In [20]:
import numpy as np
import pandas as pd
##_____________________________

def get_ds_infos():
    ## 0:Code, 1:Weight, 2:Height, 3:Age, 4:Gender
#     dataX = "/Users/g-nattatorn/Documents/Thesis/Github/motion-sense-master/data/data_subjects_info.csv"

    ##get data that relate to demonstrator (Weight, Height, Age, Gender)
    dss = np.genfromtxt("user-data-info.csv",delimiter=',')
    ##print("dss",dss)
    ##Drop header or delete nan, [1:] is the first column is not used.
    dss = dss[1:]
    ##print("dss",dss)
    ##print("----> Data subjects information is imported.")
    return dss
##print(get_ds_infos())
##____________
def creat_time_series(num_features, num_act_labels, num_gen_labels, label_codes, trial_codes):
    dataset_columns = num_features+num_act_labels+num_gen_labels
    ##print("dataset columns: ",dataset_columns)
    ds_list = get_ds_infos()
    ## print("ds list",ds_list[:,0])
    ## call function get_ds_infos
    ## create tran and test parameter blank array for keep data
    train_data = np.zeros((0,dataset_columns))
    test_data = np.zeros((0,dataset_columns))
    ## sub_id relate to sensor data
    ## [:,0] this notation will give all the values in column = 0
    for i, sub_id in enumerate(ds_list[:,0]):
        ## act relate to activity
        ##print("i or sub_id",i) ## print number 1-24
        for j, act in enumerate(label_codes):
            ##print(trial_codes[act])
            for trial in trial_codes[act]:
                ##print("trial", trial)
                fname = '/Users/g-nattatorn/Documents/Thesis/PoC-AutoEncoder/DeviceMotion-data/'+act+'_'+str(trial)+'/sub_'+str(int(sub_id))+'.csv'
                ##print(str(int(sub_id)))
                raw_data = pd.read_csv(fname)
                raw_data = raw_data.drop(['Unnamed: 0'], axis=1)
                ## print(raw_data.values) it has only value
                ## print(raw_data)
                unlabel_data = raw_data.values
                ## print(unlabel_data)
                label_data = np.zeros((len(unlabel_data), dataset_columns))
                ## print("len",(len(unlabel_data)))
                ## label_data is set all value in metric to 0
                ## print(label_data)
                ## print(dataset_columns) = 17
                ## [start:stop:step]
                label_data[:,:-(num_act_labels + num_gen_labels)] = unlabel_data
                ## parameter that add and it negative is unlabel data
                ## CNN will check that what type of parameter?
                print(unlabel_data)
                label_data[:,label_codes[act]] = 1
                ## print(label_data[:,label_codes[act]]) put 1
                label_data[:,-(num_gen_labels)] = int(ds_list[i,4])
                ## We consider long trials as training dataset and short trials as test dataset
                if trial > 10:
                    test_data = np.append(test_data, label_data, axis = 0)
                    ## print("test: ",test_data)
                else:    
                    train_data = np.append(train_data, label_data, axis = 0)
                    ## print("train: ",train_data)
    return test_data, train_data
#________________________________

## Here we set parameter to build labeld time-series from dataset of DeviceMotion_data"
num_features = 12 # attitude(roll, pitch, yaw); gravity(x, y, z); rotationRate(x, y, z); userAcceleration(x,y,z)
num_act_labels = 4 # dws, ups, wlk, jog
## Gender
num_gen_labels = 1 # 0/1(female/male)
## Look at the paper
label_codes = {"meal":num_features, "household":num_features+1, "work":num_features+2, "travel":num_features+3}
##print(label_codes)
trial_codes = {"meal":[1,2,11], "household":[3,4,12], "work":[7,8,15], "travel":[9,16]}
##print(trial_codes)
## Calling 'creat_time_series()' to build time-series
train_ts, test_ts = creat_time_series(num_features, num_act_labels, num_gen_labels, label_codes, trial_codes)
print("--> Training Time-Seires:", train_ts)
print("--> Test Time-Series:", test_ts)

[[ 1.528132 -0.733896  0.696372 ...  0.294894 -0.184493  0.377542]
 [ 1.527992 -0.716987  0.677762 ...  0.219405  0.035846  0.114866]
 [ 1.527765 -0.706999  0.670951 ...  0.010714  0.134701 -0.167808]
 ...
 [ 1.830821 -0.578367  2.447967 ... -0.08559  -0.030209 -0.08774 ]
 [ 1.849557 -0.586962  2.439458 ... -0.048105  0.029555  0.060441]
 [ 1.869375 -0.596783  2.433775 ... -0.065011 -0.042575  0.046052]]
[[ 0.05527  -0.954541 -0.609278 ... -0.432808 -0.107629  0.338057]
 [ 0.034557 -0.942493 -0.60696  ... -0.450254 -0.134344  0.312718]
 [ 0.016687 -0.959855 -0.610954 ... -0.206303 -0.353853  0.164288]
 ...
 [ 0.217096 -1.11855   0.110354 ... -0.143594 -0.379677  0.373187]
 [ 0.174744 -1.098253  0.11635  ... -0.197951 -0.490908  0.272974]
 [ 0.136041 -1.092183  0.111645 ... -0.215369 -0.585434  0.187922]]
[[ 0.086504 -1.172129  1.18166  ... -0.067855 -0.048792  0.413061]
 [ 0.059454 -1.146609  1.248254 ... -0.160289 -0.187298  0.645487]
 [ 0.029339 -1.13004   1.295114 ... -0.2177   -0.2

In [24]:
def time_series_to_section(dataset, num_act_labels, num_gen_labels, sliding_window_size, step_size_of_sliding_window, standardize = False, **options):
    print("dataset", dataset)
    ## dataset equal to train and test data set in function creat_time_series
    data = dataset[: , 0:-(num_act_labels+num_gen_labels)]
    ##print("data", data)
    ##data = dataset[0:-(num_act_labels+num_gen_labels)]
    ##print("data:", dataset[:, 0:-(num_act_labels+num_gen_labels)])
    act_labels = dataset[: , -(num_act_labels+num_gen_labels):-(num_gen_labels)]
    ##print("act labels", act_labels)
    gen_labels = dataset[: , -(num_gen_labels)]
    ##print("gen labels", gen_labels)
    mean = 0
    std = 1
    if standardize:
        ## Standardize each sensor’s data to have a zero mean and unity standard deviation.
        ## As usual, we normalize test dataset by training dataset's parameters 
        if options:
            mean = options.get("mean")
            print("mean", options.get("mean"))
            std = options.get("std")
            print("----> Test Data has been standardized")
        else:
            mean = data.mean(axis=0)
            std = data.std(axis=0)
            print("----> Training Data has been standardized:\n the mean is = ",str(mean.mean())," ; and the std is = ",str(std.mean()))            
        data -= mean
        ## print("data mean", data)
        data /= std
        ## print("data mean", data)
    else:
        print("----> Without Standardization.....")
    ## We want the Rows of matrices show each Feature and the Columns show time points.
    data = data.T
    ##print("data.t", data)
    size_features = data.shape[0]
    ## relate to num_features it has 12
    ##print("size_features ",size_features)
    size_data = data.shape[1]
    ##print("size_data", size_data)
    number_of_secs = round(((size_data - sliding_window_size)/step_size_of_sliding_window))
    ##print("number_of_secs", number_of_secs)
            
    ##  Create a 3D matrix for Storing Snapshots  
    secs_data = np.zeros((number_of_secs , size_features , sliding_window_size ))
    ## secs data [0,0,0,0]
    ## print("secs_data", secs_data)
    act_secs_labels = np.zeros((number_of_secs, num_act_labels))
    ## act_secs_labels [0.0.0.0]
    ## print("act_secs_labels", act_secs_labels)
    gen_secs_labels = np.zeros(number_of_secs)
    ## print("gen_secs_labels", gen_secs_labels)
    k=0    
    for i in range(0 ,(size_data)-sliding_window_size  , step_size_of_sliding_window):
        j = i // step_size_of_sliding_window
        if(j>=number_of_secs):
            break
        if(gen_labels[i] != gen_labels[i+sliding_window_size-1]): 
            continue
        if(not (act_labels[i] == act_labels[i+sliding_window_size-1]).all()): 
            continue    
        secs_data[k] = data[0:size_features, i:i+sliding_window_size]
        print("secs_data", secs_data[k])
        act_secs_labels[k] = act_labels[i].astype(int)
        gen_secs_labels[k] = gen_labels[i].astype(int)
        k = k+1
    secs_data = secs_data[0:k]
    act_secs_labels = act_secs_labels[0:k]
    gen_secs_labels = gen_secs_labels[0:k]
    
    return secs_data, act_secs_labels, gen_secs_labels, mean, std
##________________________________________________________________


## This Variable Defines the Size of Sliding Window
## ( e.g. 100 means in each snapshot we just consider 100 consecutive observations of each sensor) 
sliding_window_size = 50 # 50 Equals to 1 second for MotionSense Dataset (it is on 50Hz samplig rate)
## Here We Choose Step Size for Building Diffrent Snapshots from Time-Series Data
## ( smaller step size will increase the amount of the instances and higher computational cost may be incurred )
step_size_of_sliding_window = 10 
print("--> Sectioning Training and Test datasets: shape of each section will be: (",num_features,"x",sliding_window_size,")")
train_data, act_train_labels, gen_train_labels, train_mean, train_std = time_series_to_section(train_ts.copy(),
                                                                                               num_act_labels,
                                                                                               num_gen_labels,
                                                                                               sliding_window_size,
                                                                                               step_size_of_sliding_window,
                                                                                               standardize = True)

test_data, act_test_labels, gen_test_labels, test_mean, test_std = time_series_to_section(test_ts.copy(),
                                                                                          num_act_labels,
                                                                                          num_gen_labels,
                                                                                          sliding_window_size,
                                                                                          step_size_of_sliding_window,
                                                                                          standardize = True,
                                                                                          mean = train_mean, 
                                                                                          std = train_std)
print("--> Shape of Training Sections:", train_data.shape)
print("--> Shape of Test Sections:", test_data.shape)

--> Sectioning Training and Test datasets: shape of each section will be: ( 12 x 50 )
dataset [[ 0.086504 -1.172129  1.18166  ...  0.        0.        1.      ]
 [ 0.059454 -1.146609  1.248254 ...  0.        0.        1.      ]
 [ 0.029339 -1.13004   1.295114 ...  0.        0.        1.      ]
 ...
 [-1.301587 -1.362634 -1.783378 ...  0.        1.        0.      ]
 [-0.524913 -1.486641 -0.952944 ...  0.        1.        0.      ]
 [ 0.683482 -1.435057  0.377682 ...  0.        1.        0.      ]]
----> Training Data has been standardized:
 the mean is =  -0.01872874558070815  ; and the std is =  0.9087949248106734
gen_secs_labels [0. 0. 0. ... 0. 0. 0.]
dataset [[ 1.528132 -0.733896  0.696372 ...  0.        0.        1.      ]
 [ 1.527992 -0.716987  0.677762 ...  0.        0.        1.      ]
 [ 1.527765 -0.706999  0.670951 ...  0.        0.        1.      ]
 ...
 [-1.40376  -0.951444 -1.709745 ...  0.        1.        0.      ]
 [-1.504    -0.843886 -1.828078 ...  0.        1.        